In [26]:
# imports necessary packages
import re
import os
import pandas as pd
from datetime import datetime, timedelta

# set working directory
os.chdir('D:\Faith and Religion Stuff\Come, Follow Me')

### 18 Sep 2024

As I was studying the scriptures this morning, a few ideas came to me that I wanted to record so I could get back to my study ASAP and finish it before getting engaged in all of this:

* I want to implement a user input aspect to get the start and end dates. I can use Regex to determine whether or not the inputted date is satisfactory for my function.
* This calculator will actually be two calculators - the first will gather information about the start and end reading dates, and determine how many days need to be spent on each talk, and assign dates for each readthrough. **NOTE**: Something I'll need to also account for is whether or not I want to read multiple times. 
* The second calculator will break down each talk into sections of paragraphs based on the number of days that will be spent reading it. 

### 27 Sep 2024

Having dealt with Github desktop giving me problems (it decided out of the blue to stop having access to my D:// drive), with working through the "Dealing with Dates" module twice (once before Github desktop started giving me problems, and then once after I fixed those problems after deleting the work I had already done on the module), I'm now ready to start working on my General Conference calculator. I have:
* the talks from the most recent conference
* a list of information about all the talks
* an understanding of working with dates

I'm hoping that there isn't anything else I'll need to get this project really going now. 

In [32]:
apr_2024_info = pd.read_csv('apr_2024_gc_info.csv')
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146


One thing I know I'll need to do is establish an initial start date and end date. These are the first day I want to read any portion of any talk from this conference and the last day I want to read any portion of any talk from this conference. These two dates will help me calculate how many days should be spent on each talk, subtracting the ini_start_date from the ini_end_date will tell me how many total days I have to work with. 

In [28]:
ini_start = str(input("What day would you like to start reading?"))
ini_end = str(input("What day would you like to stop reading?"))

ini_start_date = pd.to_datetime(ini_start,format='%m/%d/%Y')
ini_end_date = pd.to_datetime(ini_end,format='%m/%d/%Y')
ini_total_days = ini_end_date - (ini_start_date - timedelta(days=1))
ini_total_days

Timedelta('177 days 00:00:00')

I set the cell above up to take a user input for the start and end dates, to save those, and then to convert those strings to datetime objects. Then I created a new object (ini_total_days) which is a timedelta object that shows the number of days between my start date and end date. In order to ensure that my calculator includes the start date, I had to subtract one day from the initial start date (`ini_start_date`). The resulting `Timedelta` object is 177 days. 

Having that 177 calculation, I can now calculate how many days each talk gets. When I initially did this using a Google Sheet, I did it by first establishing how many times I wanted to read each talk. Then, I established, based on the 177 days I had, I determined how many days would be allotted to each readthrough. From there, I allotted each talk a number of days based on the percentage of total speaking time that talk took to deliver, and added days to talks here and there to make sure there was no remainder of days.

For the sake of consistency, for this calculator I will assume, as I did before, that I will read each talk twice except President Nelson's, which I decided to read 3 times. However, I'll also want to get a user input to establish a read weight column for this calculation. 

In order to account for a possible remainder of days I'll need to establish some sort of standard method for my calculation to follow so I don't have to choose which talks get the remainder days. 

In the cell below, I first establish a read weight (ie, the number of times I want to read each talk) based on role code and organization. Basically, I tell python to look at all of the unique role codes (which represent the organization to which the speaker belongs), then ask the user how many times they want to read talks given by speakers from that organization, and saves that information to a dataframe that then gets merged into the original conference info dataframe in a way that ensures that each talk gets a read weight according to the organization the speaker belongs to. 

The reason I want to include user input when determining the read weight is because I may not want to read every talk more than once in the future, or I may want to read the entire First Presidency's talks more frequently than the others, or something like that. I want the flexibility of being able to specify in the future which organizations' talks I want to read more or less frequently, without having to adjust my code. 

In [33]:
# Initialize an empty read_weights list
read_weights = []

# for every unique role code in the role_code column of the apr_2024_info dataframe...
for item in apr_2024_info.role_code.unique():
    # ... ask the user what the read weight should be and...
    read_weight = int(input(f"How many times would you like to read talks given by the {item}?"))
    # ... save both the role code and the read weight to the read_weights list
    read_weights.append({'role_code':item, 'read_weight':read_weight})

# convert the read_weights list to a dataframe, save with the same name to replace the old item
read_weights = pd.DataFrame(read_weights)

# left-merge the read_weights dataframe to the apr_2024_info dataframe useing the role_code columns as a guide for merging
# left-merge keeps everything in the dataframe being merged to, and only merges data from the second dataframe that has a corresponding value in the original dataframe
apr_2024_info = apr_2024_info.merge(read_weights, on='role_code',how='left')

# display the updated dataframe
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2


In the cell below, I just calculate and display how many talks, paragraphs, and lines I'll need to read, given that I'll be reading each talk multiple times. The total_time also just calculates how many seconds of speaking I'll be reading. It doesn't make much sense, but I'll use it tomorrow or Monday. 

In [53]:
total_talks_read = sum(apr_2024_info['read_weight'])
total_time = sum(apr_2024_info['time']*apr_2024_info['read_weight'])
total_paragraphs = sum(apr_2024_info['paragraphs']*apr_2024_info['read_weight'])
total_lines = sum(apr_2024_info['lines']*apr_2024_info['read_weight'])

print(f'Total talks to read: {total_talks_read}\n'
      f'Total time: {round(total_time/60)} minutes\n'
      f'Total paragraphs: {total_paragraphs}\n'
      f'Total lines: {total_lines}')

Total talks to read: 65
Total time: 850 minutes
Total paragraphs: 2061
Total lines: 10934
